# BGD的非凸优化问题1：loss surface形态不好

凸函数可以收敛到唯一最优解，非凸函数则只能收敛到local min，所以非凸函数的收敛目标是：收敛快，收敛到一个相对好的local min。

<font color=norange>**model框架：**
$$
input \rightarrow \underset{optimization}{{\underbrace{\overset{f(x,w)}{\overbrace{feature\ representation + classifier}} \rightarrow \overset{Div,reg}{ \overbrace{ cost function}}}}} 
$$</font>

**整个model可以拆分为两大类问题：模型结构设计和优化求解**
1. 设计一套好的模型结构，包括选择三件套$f(x,w), Div和Reg$，使min expected Loss能尽可能接近min Error这一真实目标
2. 选择好的优化方法(optimization method)，能以高效的方式找到相对好的local min

**梯度下降是求解优化问题的标准方法，但实践中有两类典型问题要解决**：
1. loss surface形态不好带来的各种问题
2. 网络深度带来的计算问题

### I.1 凸函数condition number大可能引起oscillate和收敛慢
<font color=green>**solution：Adam = Momentum + RMSprop, input normalization**</font>
1. **oscillate** \
(1) 发生原因：单一learning rate时，为了让收敛速度不至于太慢，有的维度$η_{i, opt}<η_{i}<2*η_{i, opt}$，此时，这些维度上梯度符号会正负交互，converge trajectory会发生oscillate。\
<font color=orange>注：和fluctuate不同，oscillate的方向是对的，fluctuate方向是错的。fluctuate是SGD因为抽样的随机性使minibatch得到的梯度$\ne$偏离真实梯度方向而发生。</font> \
(2) solution: 用梯度的历史平均来决定方向，平滑掉波动，如：**Momentum，Nesterov** 
2. 收敛慢：\
(1) 发生原因：当$\eta$时scalar时，为迁就$\lambda$较大的方向而调低了$\eta$大小，使得$\lambda$较小的那些方向上的收敛速度变慢。\
(2) solution：不同的维度用不同的learning rate，adaptive learning rate。如：**RMSprop**

### I.2 非凸函数: random restart, SGD + lr decay, skip connection
1. saddle point：\
(1) 描述：在该点处gradient=0，迭代停止，但hessian的特征值不是都大于0，而是有正有负。\
(2) solution: Momentum正好解决了这个问题
2. inflection point:\
(1) 描述：在该点处gradient=0，迭代停止，但hessian的特征值也是0。在图形上是连续的“一字型平地”。\
(2) solution: Momentum一定程度上解决了这个问题
3. valley：\
(1) 描述：水流冲开山谷形成的深V形的峡谷。这种形态容易发生oscillate。\
(2) solution: adaptive learning rate
4. plateau: \
(1) 描述：A plateau is a flat-topped elevated landform从周围平缓地势上拔地而起. \
(2) solution: RMSprop可以减轻这个问题
5. poor local min \
(1) 描述：指该local min与global min差距很大 \
(2) solution：**random restart，SGD+learning rate decay**的效果更好，因为SGD寻址的范围更广
 - **learning rate decay**: 需要的假设是，开口大的位置local min更好，开口小的位置local min不好，所以一开始用大的learning rate可以escape from这些形态不好的位置，直到找到还不错的位置（error rate有了大幅下滑后），再逐步降低learning rate，在选中区域的local min位置converge
6. nice local min(指该local min与global min接近)附近的loss surface形态不好 \
(1) 描述：比如high gradient near the min，low gradient far from the min. \
(2) solution：选择好的divergence function和model 结构。如：skip connection[参考：percy liang对loss land scape的研究]